In [17]:
from config import set_environment
set_environment()

In [18]:
import logging
import sys

In [19]:
import nest_asyncio

nest_asyncio.apply()

In [20]:
from llama_index.core import Settings
from llama_index.core.response.notebook_utils import display_response, display_source_node

from llama_index.core import SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import VectorStoreIndex
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor, KeywordNodePostprocessor

from llama_index.llms.openai import OpenAI

from llama_index.core import get_response_synthesizer

In [21]:
# Various Test Docs

paul_graham = "documents/test/paul_graham_essay.txt"

aetna_policy = "test/benefits_qa_store/AetnaSBC.pdf"
company_policy = "test/benefits_qa_store/Company - SPD.pdf"


In [36]:
# Node Parser
chunk_size = 100

# Retriever Settings
similarity_top_k = 5

# Context Post Processor Settings
required_key_words = [""]
excluded_key_words = [""]
similarity_cutoff = 0.2

# Response Settings
response_mode = "tree_summarize" # "refine", "tree_summarize"

# Source Node Display Length
source_length = 200

In [23]:
documents = SimpleDirectoryReader(input_files=[paul_graham]).load_data()

In [24]:
node_parser = SentenceSplitter(chunk_size=chunk_size)
nodes = node_parser.get_nodes_from_documents(documents)
# set node ids to be a constant
for idx, node in enumerate(nodes):
    node.id_ = f"node-{idx}"

In [25]:
# loads BAAI/bge-small-en-v1.5
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

Settings.embed_model = embed_model

In [26]:
index = VectorStoreIndex(nodes, embed_model=Settings.embed_model, show_progress=True)

Generating embeddings:   0%|          | 0/21 [00:00<?, ?it/s]

In [28]:
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=similarity_top_k
)

In [29]:
node_postprocessors = [
    #KeywordNodePostprocessor(
    #   required_keywords=required_key_words, exclude_keywords=excluded_key_words
    #),
    SimilarityPostprocessor(similarity_cutoff=similarity_cutoff) 
]

In [30]:
response_synthesizer = get_response_synthesizer(response_mode = response_mode)

In [33]:
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=node_postprocessors
)

In [34]:
response = query_engine.query("who is Paul Graham")
display_response(response)

**`Final Response:`** Paul Graham is an individual known for his involvement in various ventures and projects in the tech industry. He has worked on startups like Viaweb and Y Combinator, written essays on different topics, and developed a new dialect of Lisp called Arc. Graham has a background in programming and has shown a preference for working on innovative projects throughout his career.

In [37]:
retrievals = retriever.retrieve("how much do i pay for home health care")
for n in retrievals:
    display_source_node(n, source_length=source_length)

**Node ID:** node-9<br>**Similarity:** 0.6103197296436534<br>**Text:** Which meant being easy to use and inexpensive. It was lucky for us that we were poor, because that caused us to make Viaweb even more inexpensive than we realized. We charged $100 a month for a sma...<br>

**Node ID:** node-10<br>**Similarity:** 0.532475705891528<br>**Text:** Nor had I changed my grad student lifestyle significantly since we started. So when Yahoo bought us it felt like going from rags to riches. Since we were going to California, I bought a car, a yell...<br>

**Node ID:** node-8<br>**Similarity:** 0.5318903557877852<br>**Text:** In return for that and doing the initial legal work and giving us business advice, we gave him 10% of the company. Ten years later this deal became the model for Y Combinator's. We knew founders ne...<br>

**Node ID:** node-5<br>**Similarity:** 0.5237020576110449<br>**Text:** But Interleaf still had a few years to live yet. [5]

Interleaf had done something pretty bold. Inspired by Emacs, they'd added a scripting language, and even made the scripting language a dialect ...<br>

**Node ID:** node-19<br>**Similarity:** 0.512010379905828<br>**Text:** I wrote an essay for myself to answer that question, and I was surprised how long and messy the answer turned out to be. If this surprised me, who'd lived it, then I thought perhaps it would be int...<br>

Multi Query

In [ ]:
from llama_index.llms.openai import OpenAI

In [ ]:
# LLM (gpt-3.5)
gpt35 = OpenAI(temperature=0, model="gpt-3.5-turbo")

# LLM (gpt-4)
gpt4 = OpenAI(temperature=0, model="gpt-4")

In [ ]:
from llama_index.core.indices.query.query_transform.base import (
    StepDecomposeQueryTransform,
)

In [ ]:
# gpt-4
step_decompose_transform = StepDecomposeQueryTransform(llm=gpt4, verbose=True)

In [ ]:
index_summary = "Used to answer questions about benefits coverage"

In [ ]:
# set Logging to DEBUG for more detailed outputs
from llama_index.core.query_engine import MultiStepQueryEngine

query_engine = index.as_query_engine(llm=gpt4)
query_engine = MultiStepQueryEngine(
    query_engine=query_engine,
    query_transform=step_decompose_transform,
    index_summary=index_summary,
)
response_gpt4 = query_engine.query(
    "Wwhat coverage do you offer for life, dental, and vision,"
)

print(response)
display_response(response)

In [ ]:
sub_qa = response_gpt4.metadata["sub_qa"]
tuples = [(t[0], t[1].response) for t in sub_qa]
print(tuples)

Sub Question Query

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.core import Settings

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
# Using the LlamaDebugHandler to print the trace of the sub questions
# captured by the SUB_QUESTION callback event type
llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

Settings.callback_manager = callback_manager

In [ ]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=query_engine,
        metadata=ToolMetadata(
            name="benefits questions",
            description="Question about benefits coverage",
        ),
    ),
]

query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    use_async=True,
)

In [ ]:
response = query_engine.query(
    "is aromatherpay supported"
)

In [ ]:
display_response(response)

Auto Merge Retriever

In [ ]:
from pathlib import Path

from llama_index.readers.file import PDFReader
from llama_index.readers.file import PyMuPDFReader

In [ ]:
loader = PyMuPDFReader()

docs0 = loader.load(file_path=Path("policy_storage/benefits_qa_store/Company - SPD.pdf"))

In [ ]:
from llama_index.core import Document

doc_text = "\n\n".join([d.get_content() for d in docs0])
docs = [Document(text=doc_text)]

In [ ]:
from llama_index.core.node_parser import (
    HierarchicalNodeParser,
    SentenceSplitter,
)

In [ ]:
node_parser = HierarchicalNodeParser.from_defaults()

In [ ]:
nodes = node_parser.get_nodes_from_documents(docs)

In [ ]:
from llama_index.core.node_parser import get_leaf_nodes, get_root_nodes

In [ ]:
leaf_nodes = get_leaf_nodes(nodes)
len(leaf_nodes)

In [ ]:
root_nodes = get_root_nodes(nodes)

In [ ]:
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core import StorageContext
from llama_index.llms.openai import OpenAI

In [ ]:
docstore = SimpleDocumentStore()

# insert nodes into docstore
docstore.add_documents(nodes)

# define storage context (will include vector store by default too)
storage_context = StorageContext.from_defaults(docstore=docstore)

llm = OpenAI(model="gpt-3.5-turbo")

In [ ]:
## Load index into vector index
from llama_index.core import VectorStoreIndex

base_index = VectorStoreIndex(
    leaf_nodes,
    storage_context=storage_context,
)

In [ ]:
from llama_index.core.retrievers import AutoMergingRetriever

In [ ]:
base_retriever = base_index.as_retriever(similarity_top_k=4)
retriever = AutoMergingRetriever(base_retriever, storage_context, verbose=True)

In [ ]:
# query_str = "What were some lessons learned from red-teaming?"
# query_str = "Can you tell me about the key concepts for safety finetuning"
query_str = (
    "is eye laser surgery covered by United?"
)

nodes = retriever.retrieve(query_str)
base_nodes = base_retriever.retrieve(query_str)

In [ ]:
from llama_index.core.response.notebook_utils import display_source_node

for node in nodes:
    display_source_node(node, source_length=10000)

In [ ]:
for node in base_nodes:
    display_source_node(node, source_length=10000)

In [ ]:
from llama_index.core.query_engine import RetrieverQueryEngine

In [ ]:
query_engine = RetrieverQueryEngine.from_args(retriever)
base_query_engine = RetrieverQueryEngine.from_args(base_retriever)

In [ ]:
response = query_engine.query(query_str)

In [ ]:
display_response(response)

In [ ]:
base_response = base_query_engine.query(query_str)

In [ ]:
display_response(base_response)

Large Chunk Size

In [ ]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(
    chunk_size=50000,
    chunk_overlap=20,
)

In [ ]:
loader = PyMuPDFReader()

docs0 = loader.load(file_path=Path("policy_storage/benefits_qa_store/Company - SPD.pdf"))

In [ ]:
from llama_index.core import Document

doc_text = "\n\n".join([d.get_content() for d in docs0])
docs = [Document(text=doc_text)]

In [ ]:
nodes = splitter.get_nodes_from_documents(docs)

In [ ]:
len(nodes)

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# loads BAAI/bge-small-en-v1.5
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
test_emeds = embed_model.get_text_embedding("Hello World!")

Settings.embed_model = embed_model

In [ ]:
## Load index into vector index
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex(
    nodes
)

In [ ]:
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=10,
)

In [ ]:
query_engine = RetrieverQueryEngine(
    retriever=retriever
)

In [ ]:
response = query_engine.query("is eye laser surgery covered by United")

In [ ]:
display_response(response)